<a href="https://colab.research.google.com/github/zooodung/TA-Prediction/blob/main/ta_prediction_jusung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 교통사고 위험도 예측

## 라이브러리 및 파일 호출

In [55]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [109]:
ta_data_daegu = pd.read_csv("/content/gdrive/MyDrive/WASSUP 1차 프로젝트/TA_DATA/train.csv")
ta_data_wide =  pd.read_csv("/content/gdrive/MyDrive/WASSUP 1차 프로젝트/TA_DATA/external_open/countrywide_accident.csv")
x_test = pd.read_csv("/content/gdrive/MyDrive/WASSUP 1차 프로젝트/TA_DATA/test.csv")

## 기초 통계 분석

In [100]:
ta_data_daegu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39609 entries, 0 to 39608
Data columns (total 23 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID           39609 non-null  object
 1   사고일시         39609 non-null  object
 2   요일           39609 non-null  object
 3   기상상태         39609 non-null  object
 4   시군구          39609 non-null  object
 5   도로형태         39609 non-null  object
 6   노면상태         39609 non-null  object
 7   사고유형         39609 non-null  object
 8   사고유형 - 세부분류  39609 non-null  object
 9   법규위반         39609 non-null  object
 10  가해운전자 차종     39609 non-null  object
 11  가해운전자 성별     39609 non-null  object
 12  가해운전자 연령     39609 non-null  object
 13  가해운전자 상해정도   39609 non-null  object
 14  피해운전자 차종     38618 non-null  object
 15  피해운전자 성별     38618 non-null  object
 16  피해운전자 연령     38618 non-null  object
 17  피해운전자 상해정도   38618 non-null  object
 18  사망자수         39609 non-null  int64 
 19  중상자수         39609 non-nu

In [52]:
x_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10963 entries, 0 to 10962
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      10963 non-null  object
 1   사고일시    10963 non-null  object
 2   요일      10963 non-null  object
 3   기상상태    10963 non-null  object
 4   시군구     10963 non-null  object
 5   도로형태    10963 non-null  object
 6   노면상태    10963 non-null  object
 7   사고유형    10963 non-null  object
dtypes: object(8)
memory usage: 685.3+ KB


In [38]:
ta_data_daegu.describe()

,사망자수,중상자수,경상자수,부상자수,ECLO
count,39609.000000,39609.000000,39609.000000,39609.000000,39609.000000
mean,0.007776,0.262365,1.070085,0.126865,4.726704
std,0.090109,0.500845,0.992034,0.394670,3.207206
min,0.000000,0.000000,0.000000,0.000000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,3.000000
50%,0.000000,0.000000,1.000000,0.000000,3.000000
75%,0.000000,0.000000,1.000000,0.000000,6.000000
max,2.000000,6.000000,22.000000,10.000000,74.000000


In [54]:
ta_data_daegu.describe()

,사망자수,중상자수,경상자수,부상자수,ECLO
count,39609.000000,39609.000000,39609.000000,39609.000000,39609.000000
mean,0.007776,0.262365,1.070085,0.126865,4.726704
std,0.090109,0.500845,0.992034,0.394670,3.207206
min,0.000000,0.000000,0.000000,0.000000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,3.000000
50%,0.000000,0.000000,1.000000,0.000000,3.000000
75%,0.000000,0.000000,1.000000,0.000000,6.000000
max,2.000000,6.000000,22.000000,10.000000,74.000000


In [43]:
columns = ['요일', '기상상태', '도로형태', '노면상태', '사고유형']

for column in columns:
    print(f"Column: {column}")
    print(ta_data_daegu[column].value_counts())
    print("-" * 30)

Column: 요일
금요일    6179
화요일    6023
수요일    5969
월요일    5895
목요일    5759
토요일    5670
일요일    4114
Name: 요일, dtype: int64
------------------------------
Column: 기상상태
맑음    36181
비      2627
흐림      729
기타       56
안개        8
눈         8
Name: 기상상태, dtype: int64
------------------------------
Column: 도로형태
단일로 - 기타           19485
교차로 - 교차로안          9931
교차로 - 교차로부근         5746
기타 - 기타             1986
교차로 - 교차로횡단보도내      1474
단일로 - 지하차도(도로)내      309
주차장 - 주차장            236
단일로 - 교량             232
단일로 - 고가도로위          138
단일로 - 터널              64
미분류 - 미분류              8
Name: 도로형태, dtype: int64
------------------------------
Column: 노면상태
건조       36420
젖음/습기     3100
기타          58
서리/결빙       25
침수           3
적설           3
Name: 노면상태, dtype: int64
------------------------------
Column: 사고유형
차대차     31785
차대사람     6833
차량단독      991
Name: 사고유형, dtype: int64
------------------------------


## 전처리

### 불필요한 컬럼 제거

In [110]:
columns_to_drop = ['사고유형 - 세부분류', '법규위반', '가해운전자 차종', '가해운전자 성별', '가해운전자 연령',
                   '가해운전자 상해정도', '피해운전자 차종', '피해운전자 성별', '피해운전자 연령', '피해운전자 상해정도']

ta_data_wide.drop(columns=columns_to_drop, inplace=True)
ta_data_daegu.drop(columns=columns_to_drop, inplace=True)

In [102]:
ta_data_daegu.head()

,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,사망자수,중상자수,경상자수,부상자수,ECLO
0,ACCIDENT_00000,2019-01-01 00,화요일,맑음,대구광역시 중구 대신동,단일로 - 기타,건조,차대사람,0,1,0,0,5
1,ACCIDENT_00001,2019-01-01 00,화요일,흐림,대구광역시 달서구 감삼동,단일로 - 기타,건조,차대사람,0,0,1,0,3
2,ACCIDENT_00002,2019-01-01 01,화요일,맑음,대구광역시 수성구 두산동,단일로 - 기타,건조,차대사람,0,0,1,0,3
3,ACCIDENT_00003,2019-01-01 02,화요일,맑음,대구광역시 북구 복현동,단일로 - 기타,건조,차대차,0,1,0,0,5
4,ACCIDENT_00004,2019-01-01 04,화요일,맑음,대구광역시 동구 신암동,단일로 - 기타,건조,차대차,0,0,1,0,3


### 사고일시 컬럼 연도, 월, 시간 단위로 분할

In [111]:
ta_data_daegu['사고일시'] = pd.to_datetime(ta_data_daegu['사고일시'])

ta_data_daegu['연도'] = ta_data_daegu['사고일시'].dt.year
ta_data_daegu['월'] = ta_data_daegu['사고일시'].dt.month
ta_data_daegu['시간'] = ta_data_daegu['사고일시'].dt.hour

### 요일 컬럼 평일과 주말로 분류

In [116]:
ta_data_daegu['요일 분류'] = ta_data_daegu['요일'].apply(lambda x: '평일' if x in ['월요일', '화요일', '수요일', '목요일', '금요일'] else '주말')

In [117]:
ta_data_daegu

,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,사망자수,중상자수,경상자수,부상자수,ECLO,연도,월,시간,요일 분류
0,ACCIDENT_00000,2019-01-01 00:00:00,화요일,맑음,대구광역시 중구 대신동,단일로 - 기타,건조,차대사람,0,1,0,0,5,2019,1,0,평일
1,ACCIDENT_00001,2019-01-01 00:00:00,화요일,흐림,대구광역시 달서구 감삼동,단일로 - 기타,건조,차대사람,0,0,1,0,3,2019,1,0,평일
2,ACCIDENT_00002,2019-01-01 01:00:00,화요일,맑음,대구광역시 수성구 두산동,단일로 - 기타,건조,차대사람,0,0,1,0,3,2019,1,1,평일
3,ACCIDENT_00003,2019-01-01 02:00:00,화요일,맑음,대구광역시 북구 복현동,단일로 - 기타,건조,차대차,0,1,0,0,5,2019,1,2,평일
4,ACCIDENT_00004,2019-01-01 04:00:00,화요일,맑음,대구광역시 동구 신암동,단일로 - 기타,건조,차대차,0,0,1,0,3,2019,1,4,평일
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39604,ACCIDENT_39604,2021-12-31 19:00:00,금요일,맑음,대구광역시 수성구 수성동3가,교차로 - 교차로안,건조,차대차,0,0,1,0,3,2021,12,19,평일
39605,ACCIDENT_39605,2021-12-31 19:00:00,금요일,맑음,대구광역시 달서구 상인동,단일로 - 기타,건조,차대차,0,0,1,0,3,2021,12,19,평일
39606,ACCIDENT_39606,2021-12-31 21:00:00,금요일,맑음,대구광역시 달서구 월성동,교차로 - 교차로안,건조,차대차,0,2,0,0,10,2021,12,21,평일
39607,ACCIDENT_39607,2021-12-31 22:00:00,금요일,맑음,대구광역시 달서구 장동,기타 - 기타,건조,차대차,0,0,1,0,3,2021,12,22,평일


### 수치형 데이터로 변환

#### 요일 컬럼 변환

In [76]:
#label_encoder = LabelEncoder()

#ta_data_daegu['요일_수치'] = label_encoder.fit_transform(ta_data_daegu['요일'])

#day_mapping = {'월요일': 0, '화요일': 1, '수요일': 2, '목요일': 3, '금요일': 4, '토요일': 5, '일요일': 6}

#ta_data_daegu['요일_수치'] = ta_data_daegu['요일'].map(day_mapping)

#print(day_mapping)
#print(ta_data_daegu[['요일', '요일_수치']])

{'월요일': 0, '화요일': 1, '수요일': 2, '목요일': 3, '금요일': 4, '토요일': 5, '일요일': 6}
        요일  요일_수치
0      화요일      1
1      화요일      1
2      화요일      1
3      화요일      1
4      화요일      1
...    ...    ...
39604  금요일      4
39605  금요일      4
39606  금요일      4
39607  금요일      4
39608  금요일      4

[39609 rows x 2 columns]


#### 기상상태 변환

In [77]:
ta_data_daegu['기상상태_수치'] = label_encoder.fit_transform(ta_data_daegu['기상상태'])

weather_mapping = {weather: num for num, weather in enumerate(label_encoder.classes_)}

print(weather_mapping)

print(ta_data_daegu[['기상상태', '기상상태_수치']])

{'기타': 0, '눈': 1, '맑음': 2, '비': 3, '안개': 4, '흐림': 5}
      기상상태  기상상태_수치
0       맑음        2
1       흐림        5
2       맑음        2
3       맑음        2
4       맑음        2
...    ...      ...
39604   맑음        2
39605   맑음        2
39606   맑음        2
39607   맑음        2
39608   맑음        2

[39609 rows x 2 columns]


#### 노면상태 변환

In [79]:
ta_data_daegu['노면상태_수치'] = label_encoder.fit_transform(ta_data_daegu['노면상태'])

road_condition_mapping = {condition: num for num, condition in enumerate(label_encoder.classes_)}

print(road_condition_mapping)

print(ta_data_daegu[['노면상태', '노면상태_수치']])

{'건조': 0, '기타': 1, '서리/결빙': 2, '적설': 3, '젖음/습기': 4, '침수': 5}
      노면상태  노면상태_수치
0       건조        0
1       건조        0
2       건조        0
3       건조        0
4       건조        0
...    ...      ...
39604   건조        0
39605   건조        0
39606   건조        0
39607   건조        0
39608   건조        0

[39609 rows x 2 columns]


#### 사고유형 변환

In [80]:
ta_data_daegu['사고유형_수치'] = label_encoder.fit_transform(ta_data_daegu['사고유형'])

accident_type_mapping = {accident_type: num for num, accident_type in enumerate(label_encoder.classes_)}

print(accident_type_mapping)

print(ta_data_daegu[['사고유형', '사고유형_수치']])

{'차대사람': 0, '차대차': 1, '차량단독': 2}
       사고유형  사고유형_수치
0      차대사람        0
1      차대사람        0
2      차대사람        0
3       차대차        1
4       차대차        1
...     ...      ...
39604   차대차        1
39605   차대차        1
39606   차대차        1
39607   차대차        1
39608   차대차        1

[39609 rows x 2 columns]


In [81]:
ta_data_daegu

,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,사망자수,중상자수,경상자수,부상자수,ECLO,요일_수치,기상상태_수치,노면상태_수치,사고유형_수치
0,ACCIDENT_00000,2019-01-01 00,화요일,맑음,대구광역시 중구 대신동,단일로 - 기타,건조,차대사람,0,1,0,0,5,1,2,0,0
1,ACCIDENT_00001,2019-01-01 00,화요일,흐림,대구광역시 달서구 감삼동,단일로 - 기타,건조,차대사람,0,0,1,0,3,1,5,0,0
2,ACCIDENT_00002,2019-01-01 01,화요일,맑음,대구광역시 수성구 두산동,단일로 - 기타,건조,차대사람,0,0,1,0,3,1,2,0,0
3,ACCIDENT_00003,2019-01-01 02,화요일,맑음,대구광역시 북구 복현동,단일로 - 기타,건조,차대차,0,1,0,0,5,1,2,0,1
4,ACCIDENT_00004,2019-01-01 04,화요일,맑음,대구광역시 동구 신암동,단일로 - 기타,건조,차대차,0,0,1,0,3,1,2,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39604,ACCIDENT_39604,2021-12-31 19,금요일,맑음,대구광역시 수성구 수성동3가,교차로 - 교차로안,건조,차대차,0,0,1,0,3,4,2,0,1
39605,ACCIDENT_39605,2021-12-31 19,금요일,맑음,대구광역시 달서구 상인동,단일로 - 기타,건조,차대차,0,0,1,0,3,4,2,0,1
39606,ACCIDENT_39606,2021-12-31 21,금요일,맑음,대구광역시 달서구 월성동,교차로 - 교차로안,건조,차대차,0,2,0,0,10,4,2,0,1
39607,ACCIDENT_39607,2021-12-31 22,금요일,맑음,대구광역시 달서구 장동,기타 - 기타,건조,차대차,0,0,1,0,3,4,2,0,1


In [60]:
ta_data_daegu.head()

,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,사망자수,중상자수,경상자수,부상자수,ECLO,요일_numberic
0,ACCIDENT_00000,2019-01-01 00,화요일,맑음,대구광역시 중구 대신동,단일로 - 기타,건조,차대사람,0,1,0,0,5,6
1,ACCIDENT_00001,2019-01-01 00,화요일,흐림,대구광역시 달서구 감삼동,단일로 - 기타,건조,차대사람,0,0,1,0,3,6
2,ACCIDENT_00002,2019-01-01 01,화요일,맑음,대구광역시 수성구 두산동,단일로 - 기타,건조,차대사람,0,0,1,0,3,6
3,ACCIDENT_00003,2019-01-01 02,화요일,맑음,대구광역시 북구 복현동,단일로 - 기타,건조,차대차,0,1,0,0,5,6
4,ACCIDENT_00004,2019-01-01 04,화요일,맑음,대구광역시 동구 신암동,단일로 - 기타,건조,차대차,0,0,1,0,3,6


## 결측치 처리

In [25]:
ta_data_daegu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39609 entries, 0 to 39608
Data columns (total 23 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID           39609 non-null  object
 1   사고일시         39609 non-null  object
 2   요일           39609 non-null  object
 3   기상상태         39609 non-null  object
 4   시군구          39609 non-null  object
 5   도로형태         39609 non-null  object
 6   노면상태         39609 non-null  object
 7   사고유형         39609 non-null  object
 8   사고유형 - 세부분류  39609 non-null  object
 9   법규위반         39609 non-null  object
 10  가해운전자 차종     39609 non-null  object
 11  가해운전자 성별     39609 non-null  object
 12  가해운전자 연령     39609 non-null  object
 13  가해운전자 상해정도   39609 non-null  object
 14  피해운전자 차종     38618 non-null  object
 15  피해운전자 성별     38618 non-null  object
 16  피해운전자 연령     38618 non-null  object
 17  피해운전자 상해정도   38618 non-null  object
 18  사망자수         39609 non-null  int64 
 19  중상자수         39609 non-nu

In [26]:
ta_data_daegu=ta_data_daegu.dropna(axis=0)
ta_data_daegu.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38618 entries, 0 to 39608
Data columns (total 23 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID           38618 non-null  object
 1   사고일시         38618 non-null  object
 2   요일           38618 non-null  object
 3   기상상태         38618 non-null  object
 4   시군구          38618 non-null  object
 5   도로형태         38618 non-null  object
 6   노면상태         38618 non-null  object
 7   사고유형         38618 non-null  object
 8   사고유형 - 세부분류  38618 non-null  object
 9   법규위반         38618 non-null  object
 10  가해운전자 차종     38618 non-null  object
 11  가해운전자 성별     38618 non-null  object
 12  가해운전자 연령     38618 non-null  object
 13  가해운전자 상해정도   38618 non-null  object
 14  피해운전자 차종     38618 non-null  object
 15  피해운전자 성별     38618 non-null  object
 16  피해운전자 연령     38618 non-null  object
 17  피해운전자 상해정도   38618 non-null  object
 18  사망자수         38618 non-null  int64 
 19  중상자수         38618 non-nu

In [23]:
weather_counts=ta_data_daegu.groupby('기상상태').sum()
weather_counts=weather_counts.drop('ECLO',axis=1)

<ipython-input-23-232c0dce7dae>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  weather_counts=ta_data_daegu.groupby('기상상태').sum()
